## MISO Multi-layer Perceptron (MLP) Regressor

**In order to see the results you should start the MLflow ui separately**:
1. Open a prompt/terminal and navigate to the path of this project
2. Activate the virtual environment:  
    (Windows: ```.venv\eis_data_analytics\Scripts\activate```,  
    Linux/Mac: ```.venv/eis_data_analytics/bin/activate```)
3. Now start MLflow with ```mlflow server --port 1234``` consider to add e.g.: ```--workers=16 --gunicorn-opts='--timeout 600'```
4. Open [http://127.0.0.1:1234](http://127.0.0.1:1234) in your browser

In [ ]:
%matplotlib widget 

from modules import dataset_manipulation as dm
from modules import eisplot as eisplot
from modules.eisplot import plt
from modules.eisplot import mpl

import numpy as np
import pandas as pd

from functools import partial
import hyperopt
import mlflow
import shapely
from sklearn.neural_network import MLPRegressor

from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnxruntime as rt

## if you have installed latex and want to use it for plots, uncomment the following 3 lines
# mpl.rcParams.update({"text.usetex": True,'savefig.format':'pdf'})
# mpl.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
# mpl.rc('text.latex', preamble=r'\usepackage{underscore}')

## save figures e.g. with:
# plot_name = "custom_3D_plot"
# plt.savefig(r"./figures/" + name_of_this_run + "_" + plot_name + ".pdf")
# plt.savefig(r"./figures/" + name_of_this_run + "_" + plot_name + ".png", dpi=600)

### Load Data

In [ ]:
name_of_this_run = "example_data"

destination_filepath = r"./data/eis_datasets/" + name_of_this_run + ".parquet"
df = pd.read_parquet(destination_filepath)
destination_filepath = r"./data/key_lookup/key_lookup_" + name_of_this_run + ".parquet"
key_lookup_df = pd.read_parquet(destination_filepath)

In [ ]:
# Can be any of the following:
pd.set_option("display.max_columns", None)
df.head(0)

In [ ]:
input_parameters = []
input_parameters += key_lookup_df["EIS_Z_abs"].to_list()
# input_parameters += key_lookup_df["EIS_Z_phase"].to_list()
# input_parameters.append("Voltage")
# Give it a Name (filename friendly)
input_parameters_name = "Z_abs"

### Define Output Parameters

In [ ]:
output_parameters = []
output_parameters += ["SOC"]
# Give it a Name (filename friendly)
output_parameters_name = "SOC"

In [ ]:
output_parameters = []
output_parameters += ["SOH"]
# Give it a Name (filename friendly)
output_parameters_name = "SOH"

In [ ]:
output_parameters = []
output_parameters += ["Temperature"]
# Give it a Name (filename friendly)
output_parameters_name = "Temp"

### Split up the data for training

In [ ]:
test_labels = ["Temperature"]
output_intervals_for_test = {"Temperature": [[4, 6], [34, 36]]}

# convert to training arrays
data_set = dm.get_set(
    df,
    output_parameters,
    feature_keys=input_parameters,
    validation_split=0.2,
    output_intervals_for_test=output_intervals_for_test,
    label_for_test_intervals=test_labels,
    label_name=output_parameters_name,
)
x_train, y_train = data_set["train"]
x_validation, y_validation = data_set["validation"]
x_test, y_test = data_set["test"]

# convert everything to float32
x_train = np.float32(x_train)
y_train = np.float32(y_train)
x_validation = np.float32(x_validation)
y_validation = np.float32(y_validation)
x_test = np.float32(x_test)
y_test = np.float32(y_test)

### Scale the data

In [ ]:
dm.use_arrhenius_correction = True
dm.use_arrhenius_correction_with_factor = False
dm.arrhenius_b = -15.47
dm.arrhenius_c = 1.30
# [dm.arrhenius_correction_inverse(dm.arrhenius_correction(i))
#  for i in [0.01, 0.1, 1, 10]]

In [ ]:
if dm.use_arrhenius_correction:
    x_train = dm.arrhenius_correction(x_train)
    x_validation = dm.arrhenius_correction(x_validation)
    x_test = dm.arrhenius_correction(x_test)

In [ ]:
# Select Scaler to use (Only one!)
dm.use_min_max_scaler = True
dm.use_standard_scaler = False
dm.scale_y_data = True
# [dm.inverse_min_max_scaler(dm.min_max_scaler(i, dm.x_min, dm.x_max), dm.x_min, dm.x_max)
#  for i in [0.01, 0.1, 1, 10]]
# [dm.inverse_standard_scaler(dm.standard_scaler(i, dm.x_min, dm.x_max), dm.x_min, dm.x_max)
#  for i in [0.01, 0.1, 1, 10]]
# Standard Scaler
dm.x_mean = np.mean(x_train)
dm.x_std = np.std(x_train)
dm.y_mean = np.mean(y_train)
dm.y_std = np.std(y_train)
# Min Max scaler
dm.x_min = np.min(x_train)
dm.x_max = np.max(x_train)
if dm.scale_y_data:
    dm.y_min = np.min(y_train)
    dm.y_max = np.max(y_train)
else:
    dm.y_min = np.array(0, dtype=np.float32)
    dm.y_max = np.array(1, dtype=np.float32)

In [ ]:
if dm.use_min_max_scaler:
    x_train = dm.min_max_scaler(x_train, dm.x_min, dm.x_max)
    x_validation = dm.min_max_scaler(x_validation, dm.x_min, dm.x_max)
    x_test = dm.min_max_scaler(x_test, dm.x_min, dm.x_max)
    if dm.scale_y_data:
        y_train = dm.min_max_scaler(y_train, dm.y_min, dm.y_max)
        y_validation = dm.min_max_scaler(y_validation, dm.y_min, dm.y_max)
        y_test = dm.min_max_scaler(y_test, dm.y_min, dm.y_max)
elif dm.use_standard_scaler:
    x_train = dm.standard_scaler(x_train, dm.x_mean, dm.x_std)
    x_test = dm.standard_scaler(x_test, dm.x_mean, dm.x_std)
    x_validation = dm.standard_scaler(x_validation, dm.x_mean, dm.x_std)
    if dm.scale_y_data:
        y_train = dm.standard_scaler(y_train, dm.y_mean, dm.y_std)
        y_validation = dm.standard_scaler(y_validation, dm.y_mean, dm.y_std)
        y_test = dm.standard_scaler(y_test, dm.y_mean, dm.y_std)

### Regression

In [ ]:
# Everything is logged in mlflow, create a new experiment
experiment_name = (
    name_of_this_run
    + "_"
    + "MISO_MLP_"
    + input_parameters_name
    + "_"
    + output_parameters_name
)
mlflow_exp = mlflow.set_experiment(experiment_name=experiment_name)

In [ ]:
# Define an objective to be minimized


def objective(params, experiment_id):
    default_params = {
        "log_model": True,
        "plot_diag": True,
        "log_plot_type": "png",
        "hidden_layer_sizes": (100,),
        "activation": "relu",
        "solver": "adam",
        "alpha": 0.0001,
        "batch_size": "auto",
        "learning_rate_init": 0.001,
        "max_iter": 200,
        "shuffle": True,
        "random_state": None,
        "tol": 0.001,
        "warm_start": False,
        "early_stopping": True,
        "validation_fraction": 0.1,
        "beta_1": 0.9,
        "beta_2": 0.999,
        "epsilon": 1e-08,
        "n_iter_no_change": 20,
        "use_arrhenius_correction": dm.use_arrhenius_correction,
        "use_arrhenius_correction_with_factor": dm.use_arrhenius_correction_with_factor,
        "use_min_max_scaler": dm.use_min_max_scaler,
        "use_standard_scaler": dm.use_standard_scaler,
        "use_scale_y_data": dm.scale_y_data,
        "x_mean": dm.x_mean,
        "x_std": dm.x_std,
        "y_mean": dm.y_mean,
        "y_std": dm.y_std,
        "x_min": dm.x_min,
        "x_max": dm.x_max,
        "y_min": dm.y_min,
        "y_max": dm.y_max,
    }

    default_params.update(params)
    merged_params = default_params
    dm.use_arrhenius_correction = merged_params["use_arrhenius_correction"]
    dm.use_arrhenius_correction_with_factor = merged_params[
        "use_arrhenius_correction_with_factor"
    ]
    dm.use_min_max_scaler = merged_params["use_min_max_scaler"]
    dm.use_standard_scaler = merged_params["use_standard_scaler"]
    dm.scale_y_data = merged_params["use_scale_y_data"]
    dm.x_mean = merged_params["x_mean"]
    dm.x_std = merged_params["x_std"]
    dm.y_mean = merged_params["y_mean"]
    dm.y_std = merged_params["y_std"]
    dm.x_min = merged_params["x_min"]
    dm.x_max = merged_params["x_max"]
    dm.y_min = merged_params["y_min"]
    dm.y_max = merged_params["y_max"]

    with mlflow.start_run(experiment_id=experiment_id, run_name="MLP"):
        if merged_params["log_model"]:
            mlflow.sklearn.autolog()

        mlflow.log_param("hidden_layer_sizes", merged_params["hidden_layer_sizes"])
        mlflow.log_param("activation", merged_params["activation"])
        mlflow.log_param("solver", merged_params["solver"])
        mlflow.log_param("alpha", merged_params["alpha"])
        mlflow.log_param("batch_size", merged_params["batch_size"])
        mlflow.log_param("learning_rate_init", merged_params["learning_rate_init"])
        mlflow.log_param("max_iter", merged_params["max_iter"])
        mlflow.log_param("shuffle", merged_params["shuffle"])
        mlflow.log_param("random_state", merged_params["random_state"])
        mlflow.log_param("tol", merged_params["tol"])
        mlflow.log_param("warm_start", merged_params["warm_start"])
        mlflow.log_param("early_stopping", merged_params["early_stopping"])
        mlflow.log_param("validation_fraction", merged_params["validation_fraction"])
        mlflow.log_param("beta_1", merged_params["beta_1"])
        mlflow.log_param("beta_2", merged_params["beta_2"])
        mlflow.log_param("epsilon", merged_params["epsilon"])
        mlflow.log_param("n_iter_no_change", merged_params["n_iter_no_change"])
        mlflow.log_param("arrhenius_correction", dm.use_arrhenius_correction)
        mlflow.log_param(
            "arrhenius_correction_with_factor", dm.use_arrhenius_correction_with_factor
        )
        mlflow.log_param("min_max_scaler", dm.use_min_max_scaler)
        mlflow.log_param("standard_scaler", dm.use_standard_scaler)
        mlflow.log_param("scale_y_data", dm.scale_y_data)

        model = MLPRegressor(
            hidden_layer_sizes=merged_params["hidden_layer_sizes"],
            activation=merged_params["activation"],
            solver=merged_params["solver"],
            alpha=merged_params["alpha"],
            batch_size=merged_params["batch_size"],
            learning_rate_init=merged_params["learning_rate_init"],
            max_iter=merged_params["max_iter"],
            shuffle=merged_params["shuffle"],
            random_state=merged_params["random_state"],
            tol=merged_params["tol"],
            warm_start=merged_params["warm_start"],
            early_stopping=merged_params["early_stopping"],
            validation_fraction=merged_params["validation_fraction"],
            beta_1=merged_params["beta_1"],
            beta_2=merged_params["beta_2"],
            epsilon=merged_params["epsilon"],
            n_iter_no_change=merged_params["n_iter_no_change"],
        )

        model.fit(x_train, y_train.ravel())
        model.score(x_test, y_test.ravel())

        train_maxae_temp = dm.evaluate_max_abs_error(model, x_train, y_train)
        validation_maxae_temp = dm.evaluate_max_abs_error(
            model, x_validation, y_validation
        )
        test_maxae_temp = dm.evaluate_max_abs_error(model, x_test, y_test)
        train_mse_temp = dm.evaluate_mse(model, x_train, y_train)
        validation_mse_temp = dm.evaluate_mse(model, x_validation, y_validation)
        test_mse_temp = dm.evaluate_mse(model, x_test, y_test)
        train_rmse_temp = dm.evaluate_rmse(model, x_train, y_train)
        validation_rmse_temp = dm.evaluate_rmse(model, x_validation, y_validation)
        test_rmse_temp = dm.evaluate_rmse(model, x_test, y_test)

        mlflow.log_metric("train_maxae_temp", train_maxae_temp)
        mlflow.log_metric("validation_maxae_temp", validation_maxae_temp)
        mlflow.log_metric("test_maxae_temp", test_maxae_temp)
        mlflow.log_metric("train_mse_temp", train_mse_temp)
        mlflow.log_metric("validation_mse_temp", validation_mse_temp)
        mlflow.log_metric("test_mse_temp", test_mse_temp)
        mlflow.log_metric("train_rmse_temp", train_rmse_temp)
        mlflow.log_metric("validation_rmse_temp", validation_rmse_temp)
        mlflow.log_metric("test_rmse_temp", test_rmse_temp)

        if merged_params["plot_diag"]:
            dm.plot_diag_during_fitting(
                model,
                name_of_this_run,
                output_parameters,
                x_test,
                x_train,
                x_validation,
                data_set,
                train_rmse_temp,
                validation_rmse_temp,
                test_rmse_temp,
                merged_params,
            )

        mlflow.log_metric(
            "std_rmse", np.std([train_rmse_temp, validation_rmse_temp, test_rmse_temp])
        )
        mlflow.log_metric(
            "max_rmse", np.max([train_rmse_temp, validation_rmse_temp, test_rmse_temp])
        )
        mlflow.log_metric(
            "std_times_max_rmse",
            np.std([train_rmse_temp, validation_rmse_temp, test_rmse_temp])
            * np.max([train_rmse_temp, validation_rmse_temp, test_rmse_temp]),
        )

        # fmin() minimizes the objective
        weighted_fit_result = np.max(
            [train_rmse_temp, validation_rmse_temp, test_rmse_temp]
        )

        mlflow.log_metric("weighted_fit_result", weighted_fit_result)

    return {"loss": weighted_fit_result, "status": hyperopt.STATUS_OK}

Temperature:
    48, 20 -> 2.405839 RMSE

In [ ]:
# Define the search space

search_space = hyperopt.hp.choice(
    "MLP",
    [
        {
            "log_model": hyperopt.hp.choice("log_model", [True]),
            "plot_diag": hyperopt.hp.choice("plot_diag", [True]),
            "log_plot_type": hyperopt.hp.choice("log_plot_type", ["svg"]),
            "hidden_layer_sizes": hyperopt.hp.choice(
                "hidden_layer_sizes",
                [
                    (
                        48,
                        48,
                    )
                ],
            ),
            "activation": hyperopt.hp.choice("activation", ["relu"]),
            "solver": hyperopt.hp.choice("solver", ["adam"]),
            "alpha": hyperopt.hp.loguniform("alpha", np.log(0.00000001), np.log(0.1)),
            "batch_size": hyperopt.hp.choice("batch_size", ["auto"]),
            "learning_rate_init": hyperopt.hp.loguniform(
                "learning_rate_init", np.log(0.0000001), np.log(10)
            ),
            "max_iter": hyperopt.hp.choice("max_iter", [10000]),
            "shuffle": hyperopt.hp.choice("shuffle", [True]),
            "random_state": hyperopt.hp.choice("random_state", [None]),
            "tol": hyperopt.hp.loguniform("tol", np.log(0.0000000001), np.log(0.0001)),
            "warm_start": hyperopt.hp.choice("warm_start", [False]),
            "early_stopping": hyperopt.hp.choice("early_stopping", [True]),
            "validation_fraction": hyperopt.hp.choice("validation_fraction", [0.1]),
            "beta_1": hyperopt.hp.loguniform("beta_1", np.log(1e-3), np.log(1)),
            "beta_2": hyperopt.hp.loguniform("beta_2", np.log(1e-3), np.log(1)),
            "epsilon": hyperopt.hp.loguniform("epsilon", np.log(1e-12), np.log(1e-04)),
            "n_iter_no_change": hyperopt.hp.choice("n_iter_no_change", [20]),
            "use_arrhenius_correction": hyperopt.hp.choice(
                "use_arrhenius_correction", [dm.use_arrhenius_correction]
            ),
            "use_arrhenius_correction_with_factor": hyperopt.hp.choice(
                "use_arrhenius_correction_with_factor",
                [dm.use_arrhenius_correction_with_factor],
            ),
            "use_min_max_scaler": hyperopt.hp.choice(
                "use_min_max_scaler", [dm.use_min_max_scaler]
            ),
            "use_standard_scaler": hyperopt.hp.choice(
                "use_standard_scaler", [dm.use_standard_scaler]
            ),
            "use_scale_y_data": hyperopt.hp.choice(
                "use_scale_y_data", [dm.scale_y_data]
            ),
            "x_mean": hyperopt.hp.choice("x_mean", [dm.x_mean]),
            "x_std": hyperopt.hp.choice("x_std", [dm.x_std]),
            "y_mean": hyperopt.hp.choice("y_mean", [dm.y_mean]),
            "y_std": hyperopt.hp.choice("y_std", [dm.y_std]),
            "x_min": hyperopt.hp.choice("x_min", [dm.x_min]),
            "x_max": hyperopt.hp.choice("x_max", [dm.x_max]),
            "y_min": hyperopt.hp.choice("y_min", [dm.y_min]),
            "y_max": hyperopt.hp.choice("y_max", [dm.y_max]),
        }
    ],
)

In [ ]:
# Choose an optimization type

# algo=hyperopt.tpe.suggest
algo = hyperopt.rand.suggest

#### Fit the model, you can track it in mlflow: [http://127.0.0.1:1234](http://127.0.0.1:1234)

In [ ]:
timout_in_minutes = 24 * 60
max_evals = 10

# if java is installed (only recommended under linux or wsl)
# import pyspark
# spark_trails = hyperopt.SparkTrials(parallelism=16)
# best_result = hyperopt.fmin(
#     fn=partial(objective, experiment_id=mlflow_exp.experiment_id),
#     space=search_space,
#     algo=algo,
#     max_evals=max_evals,
#     timeout=timout_in_minutes * 60,
#     trials=spark_trails,
# )
# if java is not available
best_result = hyperopt.fmin(
    fn=partial(objective, experiment_id=mlflow_exp.experiment_id),
    space=search_space,
    algo=algo,
    max_evals=max_evals,
    timeout=timout_in_minutes * 60,
)

print(hyperopt.space_eval(search_space, best_result))

### Select the best Model and validate

In [ ]:
name_of_this_run_eval = name_of_this_run

destination_filepath = r"./data/eis_datasets/" + name_of_this_run_eval + ".parquet"
df_eval = pd.read_parquet(destination_filepath)

In [ ]:
# change if needed
input_parameters_eval = input_parameters
input_parameters_name_eval = input_parameters_name
output_parameters_eval = output_parameters
output_parameters_name_eval = output_parameters_name

test_labels_eval = test_labels
output_intervals_for_test_eval = output_intervals_for_test

In [ ]:
data_set_eval = dm.get_set(
    df_eval,
    output_parameters_eval,
    feature_keys=input_parameters_eval,
    validation_split=0.2,
    output_intervals_for_test=output_intervals_for_test_eval,
    label_for_test_intervals=test_labels_eval,
    label_name=output_parameters_name_eval,
)
x_train_eval, y_train_eval = data_set_eval["train"]
x_validation_eval, y_validation_eval = data_set_eval["validation"]
x_test_eval, y_test_eval = data_set_eval["test"]

x_train_eval = np.float32(x_train_eval)
y_train_eval = np.float32(y_train_eval)
x_validation_eval = np.float32(x_validation_eval)
y_validation_eval = np.float32(y_validation_eval)
x_test_eval = np.float32(x_test_eval)
y_test_eval = np.float32(y_test_eval)

In [ ]:
if dm.use_arrhenius_correction:
    x_train_eval = dm.arrhenius_correction(x_train_eval)
    x_validation_eval = dm.arrhenius_correction(x_validation_eval)
    x_test_eval = dm.arrhenius_correction(x_test_eval)

if dm.use_min_max_scaler:
    x_train_eval = dm.min_max_scaler(x_train_eval, dm.x_min, dm.x_max)
    x_validation_eval = dm.min_max_scaler(x_validation_eval, dm.x_min, dm.x_max)
    x_test_eval = dm.min_max_scaler(x_test_eval, dm.x_min, dm.x_max)
    if dm.scale_y_data:
        y_train_eval = dm.min_max_scaler(y_train_eval, dm.y_min, dm.y_max)
        y_validation_eval = dm.min_max_scaler(y_validation_eval, dm.y_min, dm.y_max)
        y_test_eval = dm.min_max_scaler(y_test_eval, dm.y_min, dm.y_max)
elif dm.use_standard_scaler:
    x_train_eval = dm.standard_scaler(x_train_eval, dm.x_mean, dm.x_std)
    x_test_eval = dm.standard_scaler(x_test_eval, dm.x_mean, dm.x_std)
    x_validation_eval = dm.standard_scaler(x_validation_eval, dm.x_mean, dm.x_std)
    if dm.scale_y_data:
        y_train_eval = dm.standard_scaler(y_train_eval, dm.y_mean, dm.y_std)
        y_validation_eval = dm.standard_scaler(y_validation_eval, dm.y_mean, dm.y_std)
        y_test_eval = dm.standard_scaler(y_test_eval, dm.y_mean, dm.y_std)

Open [http://127.0.0.1:1234](http://127.0.0.1:1234) to select a fitted model. If you click on it, you can extract the run ID. It could look like this: "ad26474e8c324f84906c9fc501928cae"

In [ ]:
# you can choose a specific model
# logged_model = 'ad26474e8c324f84906c9fc501928cae'
# or just load the best model
client = mlflow.tracking.MlflowClient()
runs = client.search_runs(
    experiment_ids=[mlflow.get_experiment_by_name(experiment_name).experiment_id],
    order_by=["metrics.max_rmse"],
    max_results=1,
)
logged_model = runs[0].info.run_id

# Load model as a Sklearn.
run_eval = mlflow.get_run(logged_model)
loaded_model = mlflow.sklearn.load_model(run_eval.info.artifact_uri + "/model/")

train_rmse_temp = dm.evaluate_rmse(loaded_model, x_train_eval, y_train_eval)
print("Train RMSE: " + str(train_rmse_temp))
validation_rmse_temp = dm.evaluate_rmse(
    loaded_model, x_validation_eval, y_validation_eval
)
print("Validation RMSE: " + str(validation_rmse_temp))
test_rmse_temp = dm.evaluate_rmse(loaded_model, x_test_eval, y_test_eval)
print("Test RMSE: " + str(test_rmse_temp))

unique_model_name = (
    experiment_name
    + "_"
    + mlflow.get_experiment_by_name(experiment_name).experiment_id
    + "_"
    + logged_model
)

### Plot Results

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10 * eisplot.cm, 10 * eisplot.cm))

cell_list_train = list(set(data_set_eval["df_train"].index.get_level_values(0)))
y_pred_train_eval = loaded_model.predict(x_train_eval)
y_pred_train_eval = y_pred_train_eval.ravel()

cell_list_validation = list(
    set(data_set_eval["df_validation"].index.get_level_values(0))
)
y_pred_validation_eval = loaded_model.predict(x_validation_eval)
y_pred_validation_eval = y_pred_validation_eval.ravel()

cell_list_test = list(set(data_set_eval["df_test"].index.get_level_values(0)))
y_pred_test_eval = loaded_model.predict(x_test_eval)
y_pred_test_eval = y_pred_test_eval.ravel()

if dm.scale_y_data:
    if dm.use_min_max_scaler:
        y_pred_train_eval = dm.inverse_min_max_scaler(
            y_pred_train_eval, dm.y_min, dm.y_max
        )
        y_pred_validation_eval = dm.inverse_min_max_scaler(
            y_pred_validation_eval, dm.y_min, dm.y_max
        )
        y_pred_test_eval = dm.inverse_min_max_scaler(
            y_pred_test_eval, dm.y_min, dm.y_max
        )
        y_train_plot = dm.inverse_min_max_scaler(y_train_eval, dm.y_min, dm.y_max)
        y_validation_plot = dm.inverse_min_max_scaler(
            y_validation_eval, dm.y_min, dm.y_max
        )
        y_test_plot = dm.inverse_min_max_scaler(y_test_eval, dm.y_min, dm.y_max)
    elif dm.use_standard_scaler:
        y_pred_train_eval = dm.inverse_standard_scaler(
            y_pred_train_eval, dm.y_mean, dm.y_std
        )
        y_pred_validation_eval = dm.inverse_standard_scaler(
            y_pred_validation_eval, dm.y_mean, dm.y_std
        )
        y_pred_test_eval = dm.inverse_standard_scaler(
            y_pred_test_eval, dm.y_mean, dm.y_std
        )
        y_train_plot = dm.inverse_standard_scaler(y_train_eval, dm.y_mean, dm.y_std)
        y_validation_plot = dm.inverse_standard_scaler(
            y_validation_eval, dm.y_mean, dm.y_std
        )
        y_test_plot = dm.inverse_standard_scaler(y_test_eval, dm.y_mean, dm.y_std)
else:
    y_train_plot = y_train_eval
    y_validation_plot = y_validation_eval
    y_test_plot = y_test_eval

fig, ax = eisplot.setup_scatter(
    data_set,
    test_rmse_temp,
    title=False,
    legend=False,
    fig=fig,
    ax=ax,
    ax_xlabel=False,
    ax_ylabel=False,
    subplots_adjust=True,
    add_trendline=True,
    label="",
)
ax.plot(
    y_train_plot,
    y_pred_train_eval,
    ".",
    color=eisplot.rwth_colors.colors[("petrol", 100)],
    alpha=0.5,
)
ax.plot(
    y_validation_plot,
    y_pred_validation_eval,
    "1",
    color=eisplot.rwth_colors.colors[("turqoise", 100)],
    alpha=0.5,
)
ax.plot(
    y_test_plot,
    y_pred_test_eval,
    "2",
    color=eisplot.rwth_colors.colors[("blue", 100)],
    alpha=0.5,
)

ax.set_xlabel("actual: Output")
ax.set_ylabel("predicted: Output")

legend_elements = [
    mpl.lines.Line2D(
        [0], [0], color=eisplot.rwth_colors.colors[("green", 100)], label="ideal"
    ),
    mpl.lines.Line2D(
        [0],
        [0],
        marker=".",
        linestyle="",
        color=eisplot.rwth_colors.colors[("petrol", 100)],
        label="train",
        alpha=0.5,
    ),
    mpl.lines.Line2D(
        [0],
        [0],
        marker="1",
        linestyle="",
        color=eisplot.rwth_colors.colors[("turqoise", 100)],
        label="validation",
        alpha=0.5,
    ),
    mpl.lines.Line2D(
        [0],
        [0],
        marker="2",
        linestyle="",
        color=eisplot.rwth_colors.colors[("blue", 100)],
        label="test",
        alpha=0.5,
    ),
]
ax.legend(handles=legend_elements, loc="best", scatterpoints=1, prop={"size": 8})
fig.subplots_adjust(bottom=0.14, left=0.19)

### Convert, Export, Test and Validate with ONNX

In [ ]:
input_type = [("float_input", FloatTensorType([None, len(x_train[0])]))]
onnx_filename = "microcontroller_eis_network/onnx_export/" + unique_model_name + ".onnx"
onx = convert_sklearn(loaded_model, initial_types=input_type)
with open(onnx_filename, "wb") as f:
    f.write(onx.SerializeToString())

In [ ]:
sess = rt.InferenceSession(onnx_filename, providers=["CPUExecutionProvider"])
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: x_test_eval})[0]
if dm.scale_y_data:
    if dm.use_min_max_scaler:
        pred_onx = dm.inverse_min_max_scaler(pred_onx, dm.y_min, dm.y_max)
        y_test_eval_ref = dm.inverse_min_max_scaler(y_test_eval, dm.y_min, dm.y_max)
    elif dm.use_standard_scaler:
        pred_onx = dm.inverse_standard_scaler(pred_onx, dm.y_mean, dm.y_std)
        y_test_eval_ref = dm.inverse_standard_scaler(y_test_eval, dm.y_mean, dm.y_std)
else:
    y_test_eval_ref = y_test_eval.copy()

diff = pred_onx.ravel() - y_test_eval_ref.ravel()
print(np.max(np.abs(diff)))
print(np.mean(diff))
print(np.std(diff))
print(np.sqrt(np.mean((diff) ** 2)))

In [ ]:
x_test_eval_float32 = x_test_eval.astype(np.float32)

pred_sklearn = loaded_model.predict(x_test_eval_float32)
pred_onx = sess.run([label_name], {input_name: x_test_eval_float32})[0].ravel()

if dm.scale_y_data:
    if dm.use_min_max_scaler:
        pred_sklearn = dm.inverse_min_max_scaler(pred_sklearn, dm.y_min, dm.y_max)
        pred_onx = dm.inverse_min_max_scaler(pred_onx, dm.y_min, dm.y_max)
    elif dm.use_standard_scaler:
        pred_sklearn = dm.inverse_standard_scaler(pred_sklearn, dm.y_mean, dm.y_std)
        pred_onx = dm.inverse_standard_scaler(pred_onx, dm.y_mean, dm.y_std)

diff = pred_sklearn.ravel() - pred_onx.ravel()
print("Max difference between scikit-learn and ONNX predictions:", np.max(np.abs(diff)))
print("Mean difference between scikit-learn and ONNX predictions:", np.mean(diff))
print("RMSE of differences:", np.sqrt(np.mean(diff**2)))

### Export the test data for the microcontroller

In [ ]:
header_path = "microcontroller_eis_network/Core/Inc/"
dm.create_test_header_file(data_set_eval, header_path, unique_model_name)

### Further comparison of different fits

In [ ]:
experiment_list = pd.DataFrame()
experiment_id = mlflow_exp.experiment_id
experiment_id

for exp in mlflow.search_experiments():
    if exp.experiment_id == experiment_id:
        experiment_tmp = mlflow.search_runs(experiment_ids=experiment_id)
        experiment_list = pd.concat([experiment_list, experiment_tmp])

experiment_list = experiment_list.reset_index(drop=True)
experiment_list.head()

In [ ]:
scatter_values = [
    experiment_list["metrics.train_maxae_temp"].values,
    experiment_list["metrics.validation_maxae_temp"].values,
    experiment_list["metrics.test_maxae_temp"].values,
    experiment_list["metrics.train_mse_temp"].values,
    experiment_list["metrics.validation_mse_temp"].values,
    experiment_list["metrics.test_mse_temp"].values,
    experiment_list["metrics.train_rmse_temp"].values,
    experiment_list["metrics.validation_rmse_temp"].values,
    experiment_list["metrics.test_rmse_temp"].values,
    experiment_list["params.hidden_layer_sizes"].values,
    experiment_list["params.activation"].values,
    experiment_list["params.solver"].values,
    experiment_list["params.alpha"].values,
    experiment_list["params.batch_size"].values,
    experiment_list["params.learning_rate_init"].values,
    experiment_list["params.max_iter"].values,
    experiment_list["params.tol"].values,
    experiment_list["params.beta_1"].values,
    experiment_list["params.beta_2"].values,
    experiment_list["params.epsilon"].values,
    experiment_list["params.n_iter_no_change"].values,
]
df_experiment = pd.DataFrame(
    np.transpose(scatter_values),
    columns=[
        "Train MAXAE in K",
        "Validation MAXAE in K",
        "Test MAXAE in K",
        "Train MSE in K^2",
        "Validation MSE in K^2",
        "Test MSE in K^2",
        "Train RMSE in K",
        "Validation RMSE in K",
        "Test RMSE in K",
        "Hidden Layer Sizes",
        "Activation",
        "Solver",
        "Alpha",
        "Batch Size",
        "Learning Rate Init",
        "Max Iter",
        "Tol",
        "Beta 1",
        "Beta 2",
        "Epsilon",
        "N Iter No Change",
    ],
)

In [ ]:
# limit the error if necessary
df_experiment[df_experiment["Test RMSE in K"] > 10] = np.nan
df_experiment = df_experiment.dropna(subset=["Test RMSE in K"])

In [ ]:
corr = df_experiment.drop(
    columns=["Hidden Layer Sizes", "Activation", "Solver", "Batch Size"]
).corr()
corr.style.background_gradient(cmap="turbo")

In [ ]:
try:
    print(unique_model_name)
except:
    unique_model_name = (
        experiment_name
        + "_"
        + mlflow.get_experiment_by_name(experiment_name).experiment_id
    )

In [ ]:
destination_filepath = r"./mlruns/" + unique_model_name + ".parquet"
experiment_list.to_parquet(destination_filepath, compression="gzip", index=True)

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(18 * eisplot.cm, 12 * eisplot.cm), sharex=True)

variables = ["Alpha", "Tol", "Beta 1", "Beta 2", "Epsilon", "Learning Rate Init"]

for variable_idx, variable in enumerate(variables):
    plot_column = np.floor(variable_idx / 2).astype("int")
    plot_row = variable_idx - 2 * plot_column

    concave_hull_ratio = 0.25

    min_error = np.min(
        [
            df_experiment["Train RMSE in K"].values,
            df_experiment["Validation RMSE in K"].values,
            df_experiment["Test RMSE in K"].values,
        ]
    )

    points = np.vstack(
        (df_experiment["Train RMSE in K"].values, df_experiment[variable].values)
    ).T.astype(np.float64)
    axs[plot_column, plot_row].scatter(
        df_experiment["Train RMSE in K"].values,
        df_experiment[variable].values,
        c=eisplot.rwth_colors.colors[("petrol", 100)],
        alpha=0.1,
        marker=".",
    )
    # points_hull = np.exp(
    #     np.array(
    #         shapely.concave_hull(
    #             shapely.MultiPoint(np.log(points)), ratio=concave_hull_ratio
    #         ).exterior.coords
    #     )
    # )
    # axs[plot_column, plot_row].fill(
    #     points_hull[:, 0],
    #     points_hull[:, 1],
    #     color=eisplot.rwth_colors.colors[("petrol", 100)],
    #     alpha=0.5,
    # )

    points = np.vstack(
        (df_experiment["Validation RMSE in K"].values, df_experiment[variable].values)
    ).T.astype(np.float64)
    axs[plot_column, plot_row].scatter(
        df_experiment["Validation RMSE in K"].values,
        df_experiment[variable].values,
        c=eisplot.rwth_colors.colors[("turqoise", 100)],
        alpha=0.1,
        marker=".",
    )
    # points_hull = np.exp(
    #     np.array(
    #         shapely.concave_hull(
    #             shapely.MultiPoint(np.log(points)), ratio=concave_hull_ratio
    #         ).exterior.coords
    #     )
    # )
    # axs[plot_column, plot_row].fill(
    #     points_hull[:, 0],
    #     points_hull[:, 1],
    #     color=eisplot.rwth_colors.colors[("turqoise", 100)],
    #     alpha=0.5,
    # )

    points = np.vstack(
        (df_experiment["Test RMSE in K"].values, df_experiment[variable].values)
    ).T.astype(np.float64)
    axs[plot_column, plot_row].scatter(
        df_experiment["Test RMSE in K"].values,
        df_experiment[variable].values,
        c=eisplot.rwth_colors.colors[("blue", 100)],
        alpha=0.1,
        marker=".",
    )
    # points_hull = np.exp(
    #     np.array(
    #         shapely.concave_hull(
    #             shapely.MultiPoint(np.log(points)), ratio=concave_hull_ratio
    #         ).exterior.coords
    #     )
    # )
    # axs[plot_column, plot_row].fill(
    #     points_hull[:, 0],
    #     points_hull[:, 1],
    #     color=eisplot.rwth_colors.colors[("blue", 100)],
    #     alpha=0.5,
    # )

    axs[plot_column, plot_row].set_ylabel(variable)
    axs[plot_column, plot_row].set_yscale("linear")
    axs[plot_column, plot_row].set_xscale("log")
    axs[plot_column, plot_row].grid()


legend_elements = [
    mpl.lines.Line2D(
        [0],
        [0],
        marker="",
        linestyle="-",
        color=eisplot.rwth_colors.colors[("petrol", 100)],
        label="train",
        alpha=0.5,
    ),
    mpl.lines.Line2D(
        [0],
        [0],
        marker="",
        linestyle="-",
        color=eisplot.rwth_colors.colors[("turqoise", 100)],
        label="validation",
        alpha=0.5,
    ),
    mpl.lines.Line2D(
        [0],
        [0],
        marker="",
        linestyle="-",
        color=eisplot.rwth_colors.colors[("blue", 100)],
        label="test",
        alpha=0.5,
    ),
    mpl.lines.Line2D(
        [0],
        [0],
        marker="",
        linestyle=":",
        color=eisplot.rwth_colors.colors[("darkred", 100)],
        label="selected value",
        alpha=1.0,
    ),
]
fig.legend(
    handles=legend_elements,
    loc="upper center",
    scatterpoints=1,
    ncol=4,
)


axs[2, 0].set_xlabel("RMSE in K")
axs[2, 1].set_xlabel("RMSE in K")
fig.tight_layout()

x_values = np.array(axs[0, 0].get_xlim()) * 0.95

axs[0, 0].plot(
    x_values,
    [float(run_eval.data.params["alpha"]), float(run_eval.data.params["alpha"])],
    linestyle=":",
    color=eisplot.rwth_colors.colors[("darkred", 100)],
)
axs[0, 1].plot(
    x_values,
    [float(run_eval.data.params["tol"]), float(run_eval.data.params["tol"])],
    linestyle=":",
    color=eisplot.rwth_colors.colors[("darkred", 100)],
)
axs[1, 0].plot(
    x_values,
    [float(run_eval.data.params["beta_1"]), float(run_eval.data.params["beta_1"])],
    linestyle=":",
    color=eisplot.rwth_colors.colors[("darkred", 100)],
)
axs[1, 1].plot(
    x_values,
    [float(run_eval.data.params["beta_2"]), float(run_eval.data.params["beta_2"])],
    linestyle=":",
    color=eisplot.rwth_colors.colors[("darkred", 100)],
)
axs[2, 0].plot(
    x_values,
    [float(run_eval.data.params["epsilon"]), float(run_eval.data.params["epsilon"])],
    linestyle=":",
    color=eisplot.rwth_colors.colors[("darkred", 100)],
)
axs[2, 1].plot(
    x_values,
    [
        float(run_eval.data.params["learning_rate_init"]),
        float(run_eval.data.params["learning_rate_init"]),
    ],
    linestyle=":",
    color=eisplot.rwth_colors.colors[("darkred", 100)],
)

fig.subplots_adjust(top=0.9)